In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io
import time
from string import digits

In [5]:
basepath='C://Users//Windows//Documents//'
data_path = basepath+"deu.txt"

In [6]:
import pandas as pd
seg= pd.DataFrame()
segments = pd.read_table(data_path,names=['source', 'target'], encoding='utf-8',index_col=False)
segments.head(50)

,source,target
0,Go.,Geh.
1,Hi.,Hallo!
2,Hi.,Grüß Gott!
3,Run!,Lauf!
4,Run.,Lauf!
5,Wow!,Potzdonner!
6,Wow!,Donnerwetter!
7,Fire!,Feuer!
8,Help!,Hilfe!
9,Help!,Zu Hülf!


In [165]:
def preprocess_sentence(sentence):
    #sentence = unicode_to_ascii(sentence.lower().strip())
    num_digits= str.maketrans('','', digits)
    
    sentence= sentence.lower()
    sentence= re.sub(" +", " ", sentence)
    sentence= re.sub("'", '', sentence)
    sentence= sentence.translate(num_digits)
    sentence= sentence.strip()
    sentence= re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = sentence.rstrip().strip()
    sentence=  'start_ ' + sentence + ' _end'
    
    return sentence

In [166]:
en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))

start_ may i borrow this book ? _end
b'start_ \xc2\xbf puedo tomar prestado este libro ? _end'


In [167]:
def new_data(path, num_examples):
    try:
        lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
        print(lines)
        word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
        print(path)
        return zip(*word_pairs)
    except ValueError as e:
        print(e)

import textblob
from textblob import TextBlob    
sample_size=60000
source, target, _ = new_data(data_path, sample_size)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=10000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



C://Users//Windows//Documents//deu.txt


In [8]:
sample_size=60000
source, target, _ = new_data(data_path, sample_size)
print(source[-1])
print(target[-1])
type(target)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=10000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



C://Users//Windows//Documents//deu.txt
start_ i wanted tom to do that . _end
start_ ich wollte ,  dass tom das tut . _end


tuple

In [8]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [38]:
source_sentence_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
source_sentence_tokenizer.fit_on_texts(source)
source_tensor = source_sentence_tokenizer.texts_to_sequences(source)
source_tensor= tf.keras.preprocessing.sequence.pad_sequences(source_tensor,padding='post' )
print(len(source_tensor))

60000


In [39]:
target_sentence_tokenizer= tf.keras.preprocessing.text.Tokenizer(filters='')
target_sentence_tokenizer.fit_on_texts(target)
target_tensor = target_sentence_tokenizer.texts_to_sequences(target)
target_tensor= tf.keras.preprocessing.sequence.pad_sequences(target_tensor,padding='post' )
print(len(target_tensor[0]))

20


In [40]:
max_target_length= max(len(t) for t in  target_tensor)
print(max_target_length)
max_source_length= max(len(t) for t in  source_tensor)
print(max_source_length)

20
13


In [41]:
source_train_tensor, source_test_tensor, target_train_tensor, target_test_tensor = train_test_split(source_tensor, target_tensor, test_size=0.2)
print(source_train_tensor)

[[   1   74 1065 ...    0    0    0]
 [   1    4   55 ...    0    0    0]
 [   1   25   57 ...    0    0    0]
 ...
 [   1   21  515 ...    0    0    0]
 [   1   21   46 ...    0    0    0]
 [   1   65   53 ...    0    0    0]]


In [42]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(source_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

48000 48000 12000 12000


In [43]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

In [44]:
print("Input Language; index to word mapping")
convert(source_sentence_tokenizer, source_train_tensor[0])
print()
print("Target Language; index to word mapping")
convert( target_sentence_tokenizer, target_train_tensor[0])

Input Language; index to word mapping
1 ----> start_
74 ----> thats
1065 ----> impossible
3 ----> .
2 ----> _end

Target Language; index to word mapping
1 ----> start_
30 ----> eso
9 ----> es
1082 ----> imposible
3 ----> .
2 ----> _end


In [45]:
BUFFER_SIZE = len(source_train_tensor)
BATCH_SIZE = 64
steps_per_epoch = len(source_train_tensor)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(source_sentence_tokenizer.word_index)+1
vocab_tar_size = len(target_sentence_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((source_train_tensor, target_train_tensor)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
type(dataset)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [46]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 13]), TensorShape([64, 20]))

In [47]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [48]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 13, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [49]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
    # hidden shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [50]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 13, 1)


In [51]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
        output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

In [53]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 15440)


In [54]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)


In [55]:
checkpoint_dir = 'training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [56]:
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([target_sentence_tokenizer.word_index['start_']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
          predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

          loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
          dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

In [57]:
steps_per_epoch

750

In [58]:
EPOCHS = 20

for epoch in range(EPOCHS):
    start = time.time()
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
        if batch % 100 == 0:
            print('Epoch {} Batch {} loss {}'.format(epoch + 1,batch, batch_loss.numpy()))
   
      
  # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 loss 3.051798105239868
Epoch 1 Batch 100 loss 1.5978341102600098
Epoch 1 Batch 200 loss 1.4779084920883179
Epoch 1 Batch 300 loss 1.4221160411834717
Epoch 1 Batch 400 loss 1.4205273389816284
Epoch 1 Batch 500 loss 1.281186580657959
Epoch 1 Batch 600 loss 1.188270092010498
Epoch 1 Batch 700 loss 1.2365596294403076
Epoch 1 Loss 1.4283
Time taken for 1 epoch 7937.8470578193665 sec

Epoch 2 Batch 0 loss 1.1294276714324951
Epoch 2 Batch 100 loss 1.1186760663986206
Epoch 2 Batch 200 loss 1.1131641864776611
Epoch 2 Batch 300 loss 1.0221192836761475
Epoch 2 Batch 400 loss 1.0021274089813232
Epoch 2 Batch 500 loss 0.9768821597099304
Epoch 2 Batch 600 loss 0.8532964587211609
Epoch 2 Batch 700 loss 0.9294888377189636
Epoch 2 Loss 1.0149
Time taken for 1 epoch 5876.921113491058 sec

Epoch 3 Batch 0 loss 0.8279016613960266
Epoch 3 Batch 100 loss 0.8308420181274414
Epoch 3 Batch 200 loss 0.8366312980651855
Epoch 3 Batch 300 loss 0.6240993142127991
Epoch 3 Batch 400 loss 0.74803936481

KeyboardInterrupt: 

In [ ]:
def evaluate(sentence):
    attention_plot = np.zeros((max_target_length, max_source_length))
    sentence = preprocess_sentence(sentence)
  #print(sentence)
  #print(source_sentence_tokenizer.word_index)

    inputs = [source_sentence_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                            maxlen=max_source_length,
                                                            padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_sentence_tokenizer.word_index['start_']], 0)

    for t in range(max_target_length):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

    # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += target_sentence_tokenizer.index_word[predicted_id] + ' '

        if target_sentence_tokenizer.index_word[predicted_id] == '_end':
            return result, sentence, attention_plot

    # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [ ]:
''''def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()'''

In [86]:
def predict(strings):
    source= TextBlob(strings)
    if source.detect_language()=='de':
        sources= (source.translate(to='eng'))
        print(sources)

In [80]:
basepath = 'C://Users//Windows//Documents//'
filename = 'german_doc_new.pdf'

In [ ]:
import os
import fitz
basepath = 'C://Users//Windows//Documents//'
filename = 'german_doc_new.pdf'
file =os.path.join(basepath,filename)
doc = fitz.open(file)
for page in doc:
    pix = page.getPixmap()
    pix.writePNG("page-%i.png" % page.number)
    zoom_x = 4.0
    zomm_y = 4.0
    mat = fitz.Matrix(zoom_x, zomm_y)
    pix = page.getPixmap(matrix = mat)
    sentence = page.getText()

In [60]:
def predict(sentence, to='eng'):
    result, sentence = evaluate(sentence)
    return result
    #attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    #plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [87]:
basepath = 'C://Users//Windows//Documents//'
filename = 'german_doc_new.pdf'

In [140]:
import os
import fitz
basepath = 'C://Users//Windows//Documents//'
filename = 'german_doc_new.pdf'
file =os.path.join(basepath,filename)
doc = fitz.open(file)
for page in doc:
    pix = page.getPixmap()
    pix.writePNG("page-%i.png" % page.number)
    zoom_x = 4.0
    zomm_y = 4.0
    mat = fitz.Matrix(zoom_x, zomm_y)
    pix = page.getPixmap(matrix = mat)
    sentence = page.getText()

In [141]:
sentence

'lebenslauf05.docx"lebenslauf05.docx" \nAgathe \nKolbe \n \nAdresse: \nKronenstraße 19, 10969 Berlin \n \nTelefon: \n033 XX XXX XX \n \nEmail: \nagathekolbe@gmail.com \n \nGeburtsdaten: \n07.12.1979  \n \nZivilstand: \nLedig, keine Kinder \nKurzprofil \nIch habe 5 Jahre Berufserfahrung, die ich in einer \ninternationalen Spedition gesammelt habe. Ich beherrsche \nfließend Englisch und Office-Software. Ich wird als energische \nund engagierte Person wahrgenommen. Ich suche einen Job \nals Sekretärin, in der ich meine Fähigkeiten und Erfahrungen \nzur Umsetzung der Unternehmensrichtlinien einsetzen werde. \nBerufliche Erfahrungen  \n05/2016 – heute \nAdministrative Assistentin bei der Flint Group GmbH in Berlin  \n• Tätigkeitsbereiche: Erstellen von Power-Point-\nPräsentationen, die zur Entwicklung des Geschäfts \nbeitrugen.  \n• Ausschreibung offener Stellen des Unternehmens auf \ngängigen Jobportalseiten.  \n• Erstellen von wöchentlichen und monatlichen \nBerichten und Präsentationen. 

In [155]:
sentence = '''Ich habe 5 Jahre Berufserfahrung, die ich in einer internationalen Spedition gesammelt habe. Ich beherrsche
fließend Englisch und Office-Software. Ich wird als energische
und engagierte Person wahrgenommen. Ich suche einen Job
als Sekretärin, in der ich meine Fähigkeiten und Erfahrungen
zur Umsetzung der Unternehmensrichtlinien einsetzen werde.'''

In [156]:
predict(sentence)

I have 5 years of professional experience that I have gained in an international freight forwarder. I speak
fluent in English and office software. I will be as energetic
and committed person. I'm looking for a job
as a secretary in which I share my skills and experience
to implement the company guidelines.


In [ ]:

I have 5 years of professional experience that I have gained in an international freight forwarder. I speak
fluent in English and office software. I will be as energetic
and committed person. I'm looking for a job
as a secretary in which I share my skills and experience
to implement the company guidelines